In [1]:
# default_exp export_named

# Export cells with a name

> Exporting `nbdev` notebook cells with a name adds the ability for a library to differentiate 

In [2]:
# export


import os
import re
from pathlib import Path

import nbdev
import nbformat
from fastcore.script import call_parse
from nbdev.export import (
    _add2all,
    _deal_import,
    _from_future_import,
    _is_external_export,
    _mk_flag_re,
    _re_blank_export,
    _re_mod_export,
    check_re,
    export_names,
    extra_add,
    find_default_export,
    get_config,
    get_nbdev_module,
    notebook2script,
    read_nb,
    save_nbdev_module,
    split_flags_and_code,
)
from nbformat import NotebookNode

In [3]:
# export

_re_named_export = _mk_flag_re(
    "exportn_[a-zA-Z0-9:_]+\W*$",
    0,
    "Matches any line with a named #exportn without any module name",
)


def extract_named_export(cell: NotebookNode):
    name = None
    tst = check_re(cell, _re_named_export)
    if tst:
        pat = re.compile(rf"""exportn_[a-zA-Z0-9:_]+\W*$""", re.MULTILINE | re.VERBOSE)
        pat_match = pat.search(tst.string).group(0).split("_")
        if len(pat_match) == 2:
            name = pat_match[1].strip()
        elif len(pat_match) > 2:
            name = "_".join(pat_match[1:]).strip()
    return name

In [4]:
nb_path = Path(os.path.join("test", "test_export.ipynb"))
nb = read_nb(nb_path)
default = find_default_export(nb["cells"])
named_exports = [ne for ne in [extract_named_export(c) for c in nb["cells"]] if ne]
assert len(named_exports) >= 3

In [5]:
cell = nbformat.v4.new_code_cell(f"#export\n some code")
assert "#export" == check_re(cell, _re_blank_export).group(0)
assert extract_named_export(cell) == None
cell = nbformat.v4.new_code_cell(f"#exportn_step:func\n some code")
assert "#exportn_step:func" == check_re(cell, _re_named_export).group(0).strip()
assert "step:func" == extract_named_export(cell)
cell = nbformat.v4.new_code_cell(f"#exportn_step:func   \n some code")
assert "#exportn_step:func" == check_re(cell, _re_named_export).group(0).strip()
assert "step:func" == extract_named_export(cell)
cell = nbformat.v4.new_code_cell(f"#exportn_step:func \n some code")
assert "#exportn_step:func" == check_re(cell, _re_named_export).group(0).strip()
assert "step:func" == extract_named_export(cell)
cell = nbformat.v4.new_code_cell(f"#exportn_step:snake_case_func \n some code")
assert (
    "#exportn_step:snake_case_func" == check_re(cell, _re_named_export).group(0).strip()
)
assert "step:snake_case_func" == extract_named_export(cell)

In [6]:
# export


def is_export_extended(cell, default):
    """Check if `cell` is to be exported and returns the name of the module 
    to export it if provided. This version adds a named export."""
    tst = check_re(cell, _re_named_export)
    if tst:
        if default is None:
            print(f"No export destination, ignored:\n{cell['source']}")
        return default, _is_external_export(tst)
    tst = check_re(cell, _re_blank_export)
    if tst:
        if default is None:
            print(f"No export destination, ignored:\n{cell['source']}")
        return default, _is_external_export(tst)
    tst = check_re(cell, _re_mod_export)
    if tst:
        return os.path.sep.join(tst.groups()[0].split(".")), _is_external_export(tst)
    else:
        return None

In [7]:
exports_extended = [e for e in [is_export_extended(c, default) for c in nb["cells"]]]
assert type(exports_extended[1][0]) == str and exports_extended[1][1]
orig = [e for e in [nbdev.export.is_export(c, default) for c in nb["cells"]] if e]
new = [e for e in [is_export_extended(c, default) for c in nb["cells"]] if e]
assert len(new) == len(orig) + 4

In [8]:
# export


def set_orig_value(
    fname: str, c: NotebookNode, a: bool, e: str, default: str, bare: bool = None
):
    if bare:
        orig = "\n"
    else:
        named_export = extract_named_export(c)
        if named_export:
            if e != default:
                named_export + f" comes from {fname.name}"
            orig = f"# {named_export}\n"
        else:
            orig = (
                f'# {"" if a else "Internal "}C'
                if e == default
                else f"# Comes from {fname.name}, c"
            ) + "ell\n"
    return orig

In [9]:
fname = str(nb_path)

In [10]:
assert any(
    c
    for c in [
        set_orig_value(fname, c, False, "test_export", "test_export")
        for c in nb["cells"]
    ]
    if c == "# step:first\n"
)

# Create a forked version of _notebook2script

In [11]:
# export


def _sciflow_notebook2script(fname, modules, silent=False, to_dict=None, bare=False):
    "Finds cells starting with `#export` and puts them into a module created by `create_mod_files`"
    bare = str(get_config().get("bare", bare)) == "True"
    if os.environ.get("IN_TEST", 0):
        return  # don't export if running tests
    sep = "\n" * (int(get_config().get("cell_spacing", "1")) + 1)
    fname = Path(fname)
    nb = read_nb(fname)
    default = find_default_export(nb["cells"])
    if default is not None:
        default = os.path.sep.join(default.split("."))
    mod = get_nbdev_module()
    exports = [is_export_extended(c, default) for c in nb["cells"]]
    cells = [
        (i, c, e) for i, (c, e) in enumerate(zip(nb["cells"], exports)) if e is not None
    ]
    for i, c, (e, a) in cells:
        if e not in modules:
            print(
                f'Warning: Exporting to "{e}.py" but this module is not part of this build'
            )
        fname_out = get_config().path("lib_path") / f"{e}.py"
        orig = set_orig_value(fname, c, a, e, default)
        flag_lines, code_lines = split_flags_and_code(c)
        code_lines = _deal_import(code_lines, fname_out)
        code = sep + orig + "\n".join(code_lines)
        names = export_names(code)
        flags = "\n".join(flag_lines)
        extra, code = extra_add(flags, code)
        code = _from_future_import(fname_out, flags, code, to_dict)
        if a:
            if to_dict is None:
                _add2all(
                    fname_out,
                    [f"'{f}'" for f in names if "." not in f and len(f) > 0] + extra,
                )
        mod.index.update({f: fname.name for f in names})
        code = re.sub(r" +$", "", code, flags=re.MULTILINE)
        if code != sep + orig[:-1]:
            if to_dict is not None:
                to_dict[e].append((i, fname, code))
            else:
                with open(fname_out, "a", encoding="utf8") as f:
                    f.write(code)
        if f"{e}.py" not in mod.modules:
            mod.modules.append(f"{e}.py")
    save_nbdev_module(mod)

    if not silent:
        print(f"Converted {fname.name}.")
    return to_dict

In [12]:
# export


@call_parse
def sciflow_build_lib():
    nbdev.export._notebook2script = _sciflow_notebook2script
    notebook2script()